In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1769871,2021-10-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1769872,2021-10-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1769873,2021-10-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1769874,2021-10-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
46091,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46093,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46095,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46097,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
46099,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1077679,2021-10-21,Arkansas,Arkansas,5001,3456,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1077681,2021-10-22,Arkansas,Arkansas,5001,3467,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1077683,2021-10-23,Arkansas,Arkansas,5001,3469,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1077685,2021-10-24,Arkansas,Arkansas,5001,3470,59.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1769871,2021-10-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1769872,2021-10-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1769873,2021-10-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1769874,2021-10-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-10-25') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
643,2021-10-25,Snohomish,Washington,64549,747.00,53061,WA,County,822083,7851.88,90.87
1284,2021-10-25,Cook,Illinois,638201,11786.00,17031,IL,County,5150233,12391.69,228.84
1924,2021-10-25,Orange,California,325123,5578.00,6059,CA,County,3175692,10237.86,175.65
2563,2021-10-25,Maricopa,Arizona,729320,12047.00,4013,AZ,County,4485414,16259.81,268.58
3202,2021-10-25,Los Angeles,California,1486473,26552.00,6037,CA,County,10039107,14806.82,264.49
...,...,...,...,...,...,...,...,...,...,...,...
1768487,2021-10-25,Wheeler,Oregon,110,1.00,41069,OR,County,1332,8258.26,75.08
1768865,2021-10-25,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00
1769212,2021-10-25,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10
1769555,2021-10-25,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
643,2021-10-25,Snohomish,Washington,64549,747.00,53061,WA,County,822083,7851.88,90.87,90.87,7851.88
1284,2021-10-25,Cook,Illinois,638201,11786.00,17031,IL,County,5150233,12391.69,228.84,228.84,12391.69
1924,2021-10-25,Orange,California,325123,5578.00,6059,CA,County,3175692,10237.86,175.65,175.65,10237.86
2563,2021-10-25,Maricopa,Arizona,729320,12047.00,4013,AZ,County,4485414,16259.81,268.58,268.58,16259.81
3202,2021-10-25,Los Angeles,California,1486473,26552.00,6037,CA,County,10039107,14806.82,264.49,264.49,14806.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768487,2021-10-25,Wheeler,Oregon,110,1.00,41069,OR,County,1332,8258.26,75.08,75.08,8258.26
1768865,2021-10-25,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00,0.00,5882.35
1769212,2021-10-25,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10,229.10,6643.76
1769555,2021-10-25,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1127220,2021-10-25,Hale,Alabama,3116,88.00,1065,AL,County,14651,21268.17,600.64,600.64,21268.17,1
1227765,2021-10-25,Clarke,Alabama,4758,84.00,1025,AL,County,23622,20142.24,355.60,355.60,20142.24,2
318324,2021-10-25,Calhoun,Alabama,22236,488.00,1015,AL,County,113605,19573.08,429.56,429.56,19573.08,3
780247,2021-10-25,Etowah,Alabama,19838,500.00,1055,AL,County,102268,19398.05,488.91,488.91,19398.05,4
951214,2021-10-25,Winston,Alabama,4522,103.00,1133,AL,County,23629,19137.50,435.91,435.91,19137.50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680761,2021-10-25,Weston,Wyoming,1041,9.00,56045,WY,County,6927,15028.15,129.93,129.93,15028.15,19
948324,2021-10-25,Johnson,Wyoming,1176,16.00,56019,WY,County,8445,13925.40,189.46,189.46,13925.40,20
1075959,2021-10-25,Sublette,Wyoming,1346,15.00,56035,WY,County,9831,13691.38,152.58,152.58,13691.38,21
1339307,2021-10-25,Lincoln,Wyoming,2705,18.00,56023,WY,County,19830,13640.95,90.77,90.77,13640.95,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
880581,2021-10-25,Lowndes,Alabama,1754,64.00,1085,AL,County,9726,18034.14,658.03,658.03,18034.14,18,1
1127220,2021-10-25,Hale,Alabama,3116,88.00,1065,AL,County,14651,21268.17,600.64,600.64,21268.17,1,2
372912,2021-10-25,Walker,Alabama,11221,370.00,1127,AL,County,63521,17665.02,582.48,582.48,17665.02,24,3
879423,2021-10-25,Crenshaw,Alabama,2453,75.00,1041,AL,County,13772,17811.50,544.58,544.58,17811.50,22,4
950058,2021-10-25,Greene,Alabama,1267,44.00,1063,AL,County,8111,15620.76,542.47,542.47,15620.76,47,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680761,2021-10-25,Weston,Wyoming,1041,9.00,56045,WY,County,6927,15028.15,129.93,129.93,15028.15,19,19
1305618,2021-10-25,Uinta,Wyoming,3794,26.00,56041,WY,County,20226,18758.03,128.55,128.55,18758.03,6,20
1339307,2021-10-25,Lincoln,Wyoming,2705,18.00,56023,WY,County,19830,13640.95,90.77,90.77,13640.95,22,21
877107,2021-10-25,Albany,Wyoming,6253,32.00,56001,WY,County,38880,16082.82,82.30,82.30,16082.82,16,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.87,7851.88,27,29
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.87,7851.88,27,29
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.87,7851.88,27,29
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.87,7851.88,27,29
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.87,7851.88,27,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765769,2021-10-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1765770,2021-10-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1765771,2021-10-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1765772,2021-10-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1122544,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21268.17,2,1,1.00,nan
1122545,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21268.17,2,1,0.00,nan
1122546,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21268.17,2,1,0.00,nan
1122547,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21268.17,2,1,0.00,nan
1122548,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,600.64,21268.17,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361991,2021-10-21,Crook,Wyoming,862,18.00,56011,WY,County,7584,11366.03,237.34,237.34,11524.26,8,23,2.00,4.07
1361992,2021-10-22,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11524.26,8,23,2.00,3.43
1361993,2021-10-23,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11524.26,8,23,0.00,3.43
1361994,2021-10-24,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11524.26,8,23,0.00,3.43


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
877055,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18034.14,1,18,1.00,4.64,0.00,nan
877056,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18034.14,1,18,0.00,4.36,0.00,nan
877057,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18034.14,1,18,0.00,3.21,0.00,nan
877058,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18034.14,1,18,0.00,2.64,0.00,nan
877059,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18034.14,1,18,0.00,2.57,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368782,2021-10-21,Teton,Wyoming,5110,14.00,56039,WY,County,23464,21778.04,59.67,59.67,21957.04,23,2,12.00,9.79,0.00,0.07
368783,2021-10-22,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,21957.04,23,2,9.00,9.64,0.00,0.07
368784,2021-10-23,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,21957.04,23,2,0.00,9.64,0.00,0.07
368785,2021-10-24,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,21957.04,23,2,0.00,9.64,0.00,0.07


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-10-25') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
445544,2021-10-25,Imperial,California,36071,765.00,6025,CA,County,181215,19905.09,422.15,422.15,19905.09,1,3,0.00,47.71,0.00,0.29
3202,2021-10-25,Los Angeles,California,1486473,26552.00,6037,CA,County,10039107,14806.82,264.49,264.49,14806.82,2,12,664.00,1008.86,3.00,15.29
221150,2021-10-25,San Bernardino,California,358677,5720.00,6071,CA,County,2180085,16452.43,262.38,262.38,16452.43,3,5,0.00,241.14,28.00,11.64
98553,2021-10-25,Stanislaus,California,86738,1338.00,6099,CA,County,550660,15751.64,242.98,242.98,15751.64,4,7,439.00,164.64,3.00,2.57
78325,2021-10-25,San Joaquin,California,102776,1745.00,6077,CA,County,762148,13485.04,228.96,228.96,13485.04,5,18,0.00,104.86,14.00,4.14
782519,2021-10-25,Inyo,California,2142,41.00,6027,CA,County,18039,11874.27,227.29,227.29,11874.27,6,24,0.00,18.29,0.00,0.14
950000,2021-10-25,Kings,California,32938,333.00,6031,CA,County,152940,21536.55,217.73,217.73,21536.55,7,2,0.00,65.93,4.00,1.79
1076465,2021-10-25,Tuolumne,California,7151,117.00,6109,CA,County,54478,13126.40,214.77,214.77,13126.40,8,22,88.00,25.50,4.00,0.79
708749,2021-10-25,Merced,California,43129,588.00,6047,CA,County,277680,15531.91,211.75,211.75,15531.91,9,8,318.00,109.29,2.00,1.29
122310,2021-10-25,Tulare,California,80375,978.00,6107,CA,County,466195,17240.64,209.78,209.78,17240.64,10,4,0.00,165.36,0.00,2.71


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1668877,2021-10-25,Lassen,California,7587,49.00,6035,CA,County,30573,24816.01,160.27,160.27,24816.01,22,1,44.00,19.21,0.00,0.71
950000,2021-10-25,Kings,California,32938,333.00,6031,CA,County,152940,21536.55,217.73,217.73,21536.55,7,2,0.00,65.93,4.00,1.79
445544,2021-10-25,Imperial,California,36071,765.00,6025,CA,County,181215,19905.09,422.15,422.15,19905.09,1,3,0.00,47.71,0.00,0.29
122310,2021-10-25,Tulare,California,80375,978.00,6107,CA,County,466195,17240.64,209.78,209.78,17240.64,10,4,0.00,165.36,0.00,2.71
221150,2021-10-25,San Bernardino,California,358677,5720.00,6071,CA,County,2180085,16452.43,262.38,262.38,16452.43,3,5,0.00,241.14,28.00,11.64
278325,2021-10-25,Kern,California,147294,1652.00,6029,CA,County,900202,16362.33,183.51,183.51,16362.33,14,6,946.00,332.64,4.00,4.29
98553,2021-10-25,Stanislaus,California,86738,1338.00,6099,CA,County,550660,15751.64,242.98,242.98,15751.64,4,7,439.00,164.64,3.00,2.57
708749,2021-10-25,Merced,California,43129,588.00,6047,CA,County,277680,15531.91,211.75,211.75,15531.91,9,8,318.00,109.29,2.00,1.29
1015310,2021-10-25,Glenn,California,4384,42.00,6021,CA,County,28393,15440.43,147.92,147.92,15440.43,25,9,0.00,9.71,0.00,0.21
44894,2021-10-25,Madera,California,23955,285.00,6039,CA,County,157327,15226.25,181.15,181.15,15226.25,15,10,0.00,51.71,0.00,0.71


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
445544,2021-10-25,Imperial,California,36071,765.00,6025,CA,County,181215,19905.09,422.15,422.15,19905.09,1,3,0.00,47.71,0.00,0.29
3202,2021-10-25,Los Angeles,California,1486473,26552.00,6037,CA,County,10039107,14806.82,264.49,264.49,14806.82,2,12,664.00,1008.86,3.00,15.29
221150,2021-10-25,San Bernardino,California,358677,5720.00,6071,CA,County,2180085,16452.43,262.38,262.38,16452.43,3,5,0.00,241.14,28.00,11.64
98553,2021-10-25,Stanislaus,California,86738,1338.00,6099,CA,County,550660,15751.64,242.98,242.98,15751.64,4,7,439.00,164.64,3.00,2.57
78325,2021-10-25,San Joaquin,California,102776,1745.00,6077,CA,County,762148,13485.04,228.96,228.96,13485.04,5,18,0.00,104.86,14.00,4.14
782519,2021-10-25,Inyo,California,2142,41.00,6027,CA,County,18039,11874.27,227.29,227.29,11874.27,6,24,0.00,18.29,0.00,0.14
950000,2021-10-25,Kings,California,32938,333.00,6031,CA,County,152940,21536.55,217.73,217.73,21536.55,7,2,0.00,65.93,4.00,1.79
1076465,2021-10-25,Tuolumne,California,7151,117.00,6109,CA,County,54478,13126.40,214.77,214.77,13126.40,8,22,88.00,25.50,4.00,0.79
708749,2021-10-25,Merced,California,43129,588.00,6047,CA,County,277680,15531.91,211.75,211.75,15531.91,9,8,318.00,109.29,2.00,1.29
122310,2021-10-25,Tulare,California,80375,978.00,6107,CA,County,466195,17240.64,209.78,209.78,17240.64,10,4,0.00,165.36,0.00,2.71


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,62.07
1,03/21/20,Imperial,4,2.21,3,0.00,60.64
2,03/22/20,Imperial,4,2.21,3,0.00,53.21
3,03/23/20,Imperial,4,2.21,3,0.00,40.93
4,03/24/20,Imperial,9,4.97,3,5.00,41.29
...,...,...,...,...,...,...,...
5800,10/21/21,Glenn,4367,15380.55,9,15.00,10.43
5801,10/22/21,Glenn,4384,15440.43,9,17.00,11.14
5802,10/23/21,Glenn,4384,15440.43,9,0.00,11.14
5803,10/24/21,Glenn,4384,15440.43,9,0.00,11.14


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.07
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.07
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.07
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.07
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.07
...,...,...,...,...,...,...,...
5906,10/21/21,Tulare,968.00,207.64,10,10.00,3.00
5907,10/22/21,Tulare,978.00,209.78,10,10.00,3.29
5908,10/23/21,Tulare,978.00,209.78,10,0.00,3.29
5909,10/24/21,Tulare,978.00,209.78,10,0.00,3.29


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)